## Ejercicio de codificación #0501

### 1. Calsificación con regresión logística:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline

#### 1.1. Leer los datos y explorar:

In [ ]:
# Cargar los datos.
data = load_breast_cancer()

In [ ]:
# Mostrar la descripción
print(data.DESCR)

In [ ]:
# Variables explicativas.
X = data['data']
print(data['feature_names'])

In [ ]:
X.shape

In [ ]:
# Variable de respuesta.
# Volver a etiquetar de modo que 0 = 'benigno' y 1 = maligno.
Y = 1 - data['target']
label = list(data['target_names'])
label.reverse()
print(label)

In [ ]:
# Mostrar la tabla de frecuencias.
ser = pd.Series(Y)
table = ser.value_counts()
table = table.sort_index()                                         # Debe ser ordenada para un etiquetado correcto.
sns.barplot(label,table.values)
plt.show()

#### 1.2. Train and test:

In [ ]:
# Dividir el conjunto de datos en entrenamiento y prueba.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=1234)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
# Entrenar y predecir.
LL = LogisticRegression(solver='liblinear',max_iter=200)
LL.fit(X_train,Y_train)
Y_pred_test = LL.predict(X_test)                            # Predicción fuera de muestra. 

In [ ]:
# Matriz de confusión
conf_mat = metrics.confusion_matrix(Y_test,Y_pred_test)
print(conf_mat)

In [ ]:
# Exactitud, Sensibilidad, Especificidad y Precisiónn usando la matriz de confusión.
accuracy = (conf_mat[0,0] + conf_mat[1,1])/np.sum(conf_mat)
sensitivity = conf_mat[1,1]/(conf_mat[1,0]+conf_mat[1,1])
specificity = conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
precision = conf_mat[1,1]/(conf_mat[0,1]+conf_mat[1,1])
print('Accuracy    = {}'.format(np.round(accuracy,3)))
print('Sensitvity  = {}'.format(np.round(sensitivity,3)))
print('Specificity = {}'.format(np.round(specificity,3)))
print('Precision   = {}'.format(np.round(precision,3)))

In [ ]:
# Modo alternativo.
accuracy = metrics.accuracy_score(Y_test,Y_pred_test)                      # Modo alternativo de calcular la exactitud.
sensitivity = metrics.recall_score(Y_test,Y_pred_test)
precision = metrics.precision_score(Y_test,Y_pred_test)
print('Accuracy    = {}'.format(np.round(accuracy,3)))
print('Sensitvity  = {}'.format(np.round(sensitivity,3)))
print('Precision   = {}'.format(np.round(precision,3)))

#### 1.3. Corte (umbral):

In [ ]:
# Ahora, predice la probabilidad de Y = 1.
Y_pred_test_prob=LL.predict_proba(X_test)[:,1]

In [ ]:
# Se puede cambiar el corte a voluntad
cutoff = 0.7                                                      # el corte puede ser un valor entre 0 y 1.
Y_pred_test_val = (Y_pred_test_prob > cutoff).astype(int)
conf_mat = metrics.confusion_matrix(Y_test,Y_pred_test_val)
print(conf_mat)

In [ ]:
accuracy = (conf_mat[0,0] + conf_mat[1,1])/np.sum(conf_mat)
sensitivity = conf_mat[1,1]/(conf_mat[1,0]+conf_mat[1,1])
specificity = conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
precision = conf_mat[1,1]/(conf_mat[0,1]+conf_mat[1,1])
print('Accuracy    = {}'.format(np.round(accuracy,3)))
print('Sensitvity  = {}'.format(np.round(sensitivity,3)))
print('Specificity = {}'.format(np.round(specificity,3)))
print('Precision   = {}'.format(np.round(precision,3)))

#### 1.4. Curva ROC:

In [ ]:
# Inicializar.
cutoff_grid = np.linspace(0.0,1.0,100)
TPR = []                                                   # Ratio de Ciertos Positivos.
FPR = []                                                   # Ratio de Falsos Positivos.

In [ ]:
# Poblar las listas de TP y FP.
for cutoff in cutoff_grid:
    Y_pred_test_val = (Y_pred_test_prob > cutoff).astype(int)
    conf_mat = metrics.confusion_matrix(Y_test,Y_pred_test_val)
    sensitivity = conf_mat[1,1]/(conf_mat[1,0]+conf_mat[1,1])
    specificity = conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
    TPR.append(sensitivity)
    FPR.append(1-specificity)                             

In [ ]:
# Visualizar.
plt.plot(FPR,TPR,c='red',linewidth=1.0)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

#### 1.5. Curva ROC (sklearn):

In [ ]:
# Calcular TPR y FPR usando una función de Scikit.
FPR, TPR, cutoffs = metrics.roc_curve(Y_test,Y_pred_test_prob,pos_label=1)      # etiqueta positiva = 1.

In [ ]:
# Visualizar.
plt.plot(FPR,TPR,c='red',linewidth=1.0)
plt.xlabel('False Positive')
plt.ylabel('True Positive')
plt.title('ROC Curve')
plt.show()

In [ ]:
# AUC.
auc = metrics.roc_auc_score(Y_test,Y_pred_test_prob)
print('AUC  = {}'.format(np.round(auc,3)))